In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!git clone https://github.com/IBM/AIX360
!pip install ./AIX360/

Cloning into 'AIX360'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 1348 (delta 15), reused 9 (delta 2), pack-reused 1304
Receiving objects: 100% (1348/1348), 357.23 MiB | 13.16 MiB/s, done.
Resolving deltas: 100% (747/747), done.
Processing ./AIX360
     |████████████████████████████████| 109.2MB 41kB/s 
     |████████████████████████████████| 109.8MB 28kB/s 
     |████████████████████████████████| 276kB 42.3MB/s 
     |████████████████████████████████| 266kB 40.2MB/s 
     |████████████████████████████████| 491kB 41.0MB/s 
     |████████████████████████████████| 3.2MB 32.9MB/s 
  Created wheel for aix360: filename=aix360-0.2.0-cp36-none-any.whl size=58397341 sha256=71ea80670b946ba2c63d4034e3667df94d2b02b2bb12f3d64688d5b3487d6828
  Stored in directory: /tmp/pip-ephem-wheel-cache-r669f1st/wheels/e1/45/37/749bf7f76b28af881e06780e87624f8cb542217ac07c670e82
  Created wheel for qpsolver

In [ ]:
import sys
sys.path.append("../../")
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn

import pandas as pd
from IPython.display import Image
import numpy as np
import math
import os
import argparse
import matplotlib.pyplot as plt
import json

from aix360.algorithms.dipvae.dipvae_utils import VAE, DIPVAE
from aix360.algorithms.dipvae import DIPVAEExplainer

In [ ]:
cuda_available = torch.cuda.is_available()
print("CUDA: {}".format(cuda_available))

base_path = '/content/drive/My Drive/Datascience1/own-model/'
print(base_path)

CUDA: True
/content/drive/My Drive/Datascience1/own-model/


In [ ]:
import torch
from torchvision import transforms
import torch.nn as nn

import pandas as pd
from PIL import Image
import numpy as np
import math
import os

import matplotlib.pyplot as plt

from aix360.algorithms.dipvae.dipvae_utils import VAE, DIPVAE
from aix360.algorithms.dipvae import DIPVAEExplainer
from aix360.algorithms.dipvae.dipvae_utils import plot_reconstructions, plot_latent_traversal

In [ ]:
class FlowersImages(torch.utils.data.Dataset):
    def __init__(self, root, file_path, transform=None):
        self.root = root
        self.data  = pd.read_csv(file_path)

        print(self.data)

        self.data['disease_class'] = self.data.apply(lambda x: self.data.columns[x==1], axis = 1)
        self.data['disease_class'] = self.data['disease_class'].apply(lambda x: str(x[0]))

        diseases = {'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'nv2': 6, 'nv3': 7, 'vasc': 8}
        self.data['disease_class_int'] = self.data['disease_class'].map(diseases)

        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path = self.root + self.data.iloc[index, 1]
        img = Image.open(img_path).convert("RGB")
        
        label = self.data.iloc[index, 10]

        if self.transform is not None:
            img = self.transform(img)
            #imghsv = self.transform(imghsv)

        #img_6channel = np.vstack((img, imghsv))

        return img, label #img_6channel


class FlowersDataset():
    def __init__(self, batch_size=256,
                 root_images_path="/content/drive/My Drive/Dataset_skin_cancer",
                 file_path_labels='/content/drive/My Drive/Datascience1/disease_classes.csv'):
        img_size = (128, 128)  # (224,224)

        # Transforms
        transform = transforms.Compose([
            transforms.Resize(size=img_size),
            transforms.ToTensor()

        ])

        train_set = FlowersImages(root=root_images_path, file_path=file_path_labels, transform=transform)

        self.train_loader = torch.utils.data.DataLoader(
            dataset=train_set,
            batch_size=batch_size,
            shuffle=True)

        self.name = "flowers"
        self.data_dims = [3, 128, 128]
        self.train_size = len(self.train_loader)
        self.range = [-1.0, 1.0]
        self.batch_size = batch_size
        self.num_training_instances = len(train_set)
        self.likelihood_type = "gaussian"
        self.output_activation_type = "tanh"

    def next_batch(self):
        for x, y in self.train_loader:
            x = 2.0 * (x - 0.5)
            yield x, y

class ConvEncNet(nn.Module):
    def __init__(self, latent_dim=20, num_filters=64, num_channels=3, image_size=128, activation_type='relu', args=None):
        super(ConvEncNet, self).__init__()
        self.args = args
        if activation_type == 'relu':
            self.activation = nn.ReLU(inplace=True)
        elif activation_type == 'tanh':
            self.activation = nn.Tanh(inplace=True)
        else:
            print("Activation Type not supported")
            return

        self.conv_hidden = []
        self.conv1 = nn.Conv2d(num_channels, num_filters, 4, 2, 1, bias=True)

        num_layers = math.log2(image_size)
        assert num_layers == round(num_layers), 'Image size that are power of 2 are supported.'
        num_layers = int(num_layers)

        for i in np.arange(num_layers - 3):
            self.conv_hidden.append(nn.Conv2d(num_filters * 2 ** i, num_filters * 2 ** (i + 1), 4, 2, 1, bias=True))
            self.conv_hidden.append(nn.BatchNorm2d(num_filters * 2 ** (i + 1)))
            self.conv_hidden.append(self.activation)

        self.features = nn.Sequential(*self.conv_hidden)
        self.conv_mu = nn.Conv2d(num_filters * 2 ** (num_layers - 3), latent_dim, 4)
        self.conv_var = nn.Conv2d(num_filters * 2 ** (num_layers - 3), latent_dim, 4)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    def forward(self, x):
        x = self.conv1(x)
        x = self.activation(x)
        x = self.features(x)
        mu_z = self.conv_mu(x).view(x.size(0), -1)
        var_z = self.conv_var(x).view(x.size(0), -1)
        return torch.cat([mu_z, var_z], dim=1)


class ConvDecNet(nn.Module):
    def __init__(self, latent_dim=20, num_filters=64, num_channels=3, image_size=128, activation_type='relu', args=None):
        super(ConvDecNet, self).__init__()
        self.args = args
        if activation_type == 'relu':
            self.activation = nn.ReLU(inplace=True)
        elif activation_type == 'tanh':
            self.activation = nn.Tanh(inplace=True)
        else:
            print("Activation Type not supported")
            return

        self.deconv_hidden = []

        num_layers = math.log2(image_size)
        assert num_layers == round(num_layers), 'Image size that are power of 2 are supported.'
        num_layers = int(num_layers)

        self.deconv1 = nn.ConvTranspose2d(latent_dim, num_filters * 2 ** (num_layers - 3), 4, 1, 0, bias=True)
        self.bn1 = nn.BatchNorm2d(num_filters * 2 ** (num_layers - 3))

        for i in np.arange(num_layers - 3, 0, -1):
            self.deconv_hidden.append(nn.ConvTranspose2d(num_filters * 2 ** i,
                                                         num_filters * 2 ** (i - 1),
                                                         4, 2, 1, bias=True))
            self.deconv_hidden.append(nn.BatchNorm2d(num_filters * 2 ** (i - 1)))
            self.deconv_hidden.append(self.activation)

        self.features = nn.Sequential(*self.deconv_hidden)

        self.deconv_out = nn.ConvTranspose2d(num_filters, num_channels, 4, 2,1, bias=True)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    def forward(self, x):
        x = x.unsqueeze(-1).unsqueeze(-1)
        x = self.deconv1(x)
        x = self.bn1(x)
        x = self.activation(x)
        x = self.features(x)
        return self.deconv_out(x)


class ConvDIPVAE(DIPVAE):
    """
    """
    def __init__(self, num_filters=50, latent_dim=10, num_channels=3, image_size=128, activation_type='relu',
                 args=None, cuda_available=None, init_networks=True, mode=None, output_activation_type='tanh',
                 likelihood_type=None, beta=1.0):
        super(ConvDIPVAE, self).__init__(latent_dim=latent_dim,
                                     activation_type=activation_type,
                                     args=args, cuda_available=cuda_available, init_networks=False, mode=mode,
                                     output_activation_type=output_activation_type,likelihood_type=likelihood_type,
                                         beta=beta)

        if init_networks:
            self.generative_net = ConvDecNet(latent_dim=latent_dim, num_filters=num_filters, num_channels=num_channels,
                                             image_size=image_size,activation_type=activation_type, args=args)
            self.inference_net = ConvEncNet(latent_dim=latent_dim, num_filters=num_filters, num_channels=num_channels,
                                            image_size=image_size, activation_type=activation_type, args=args)

        self.name = "ConvDIPVAE"


dataset_obj = FlowersDataset(root_images_path="/content/drive/My Drive/Dataset_skin_cancer/",
                          file_path_labels="/content/drive/My Drive/Datascience1/disease_classes.csv",
                          batch_size=32)


         0  ISIC_0031609.jpg  1.0  0.0  ...  0.0.4  0.0.5  0.0.6  0.0.7
0        1  ISIC_0029659.jpg  1.0  0.0  ...    0.0    0.0    0.0    0.0
1        2  ISIC_0027802.jpg  1.0  0.0  ...    0.0    0.0    0.0    0.0
2        3  ISIC_0028335.jpg  1.0  0.0  ...    0.0    0.0    0.0    0.0
3        4  ISIC_0028763.jpg  1.0  0.0  ...    0.0    0.0    0.0    0.0
4        5  ISIC_0026702.jpg  1.0  0.0  ...    0.0    0.0    0.0    0.0
...    ...               ...  ...  ...  ...    ...    ...    ...    ...
10174  509  ISIC_0026766.jpg  0.0  1.0  ...    0.0    0.0    0.0    0.0
10175  510  ISIC_0027865.jpg  0.0  1.0  ...    0.0    0.0    0.0    0.0
10176  511  ISIC_0027851.jpg  0.0  1.0  ...    0.0    0.0    0.0    0.0
10177  512  ISIC_0026091.jpg  0.0  1.0  ...    0.0    0.0    0.0    0.0
10178  513  ISIC_0030230.jpg  0.0  1.0  ...    0.0    0.0    0.0    0.0

[10179 rows x 11 columns]


In [ ]:
dipvaeii_model_args = argparse.Namespace()

dipvaeii_model_args.model = 'user-defined'
dipvaeii_model_args.activation_type = 'relu'
dipvaeii_model_args.num_filters = 32
dipvaeii_model_args.latent_dim = 10
dipvaeii_model_args.num_channels =  dataset_obj.data_dims[0]
dipvaeii_model_args.image_size =  dataset_obj.data_dims[-1]
dipvaeii_model_args.step_size = 0.0001
dipvaeii_model_args.num_epochs = 10
dipvaeii_model_args.lambda_diag_factor = 10.0
dipvaeii_model_args.lambda_offdiag = 0.001
dipvaeii_model_args.output_activation_type = dataset_obj.output_activation_type
dipvaeii_model_args.likelihood_type = dataset_obj.likelihood_type
dipvaeii_model_args.mode='ii'
dipvaeii_model_args.beta=0.001
dipvaeii_model_args.seed=0

dipvaeii_model_args.fit=1
dipvaeii_model_args.root_save_dir = base_path

# Baseline VAE where lambda_diag_factor and lambda_offdiag are set to 0.0

vae_model_args = argparse.Namespace()

vae_model_args.model = 'user-defined'
vae_model_args.activation_type = 'relu'
vae_model_args.num_filters = 32
vae_model_args.latent_dim = 10
vae_model_args.num_channels =  dataset_obj.data_dims[0]
vae_model_args.image_size =  dataset_obj.data_dims[-1]
vae_model_args.step_size = 0.0001
vae_model_args.num_epochs = 80
vae_model_args.lambda_diag_factor = 0.0
vae_model_args.lambda_offdiag = 0.0
vae_model_args.output_activation_type = dataset_obj.output_activation_type
vae_model_args.likelihood_type = dataset_obj.likelihood_type
vae_model_args.mode='ii'
vae_model_args.beta=0.001
vae_model_args.seed=0

vae_model_args.fit=1
vae_model_args.root_save_dir = base_path

In [ ]:

# Train model
model_args = dipvaeii_model_args

setup = [
    ('model={:s}', model_args.model),
    ('lambda_diag_factor={:.0e}', model_args.lambda_diag_factor),
    ('lambda_offdiag={:.0e}', model_args.lambda_offdiag),
    ('beta={:.0e}', model_args.beta),
]
save_dir = os.path.join(model_args.root_save_dir, "results"+'_'.join([t.format(v) for (t, v) in setup]))


if model_args.fit:
    net = ConvDIPVAE(num_filters=model_args.num_filters, latent_dim=model_args.latent_dim,
                  num_channels=model_args.num_channels, image_size=model_args.image_size,
                  activation_type=model_args.activation_type, args=model_args, cuda_available=cuda_available,
                  init_networks=True, mode=model_args.mode, output_activation_type=model_args.output_activation_type,
                  likelihood_type=model_args.likelihood_type, beta=model_args.beta)
else:
    net = torch.load(os.path.join(save_dir, net))

dipvaeii_explainer = DIPVAEExplainer(net=net, dataset=dataset_obj, cuda_available=cuda_available,
                                      model_args=model_args)

if model_args.fit:
    loss_epoch_list = dipvaeii_explainer.fit(visualize=True, save_dir=save_dir)
    torch.save(net, os.path.join(save_dir, net))

# After training
for x, _ in dataset_obj.next_batch():
    if dipvaeii_explainer.cuda_available:
        x = x.cuda()
    plot_reconstructions(dipvaeii_explainer.dataset, dipvaeii_explainer.net, x, image_id_to_plot=2,
                          epoch='end', batch_id = 'end', save_dir=save_dir)
    plot_latent_traversal(dipvaeii_explainer, x, dipvaeii_explainer.model_args, dipvaeii_explainer.dataset,
                          image_id_to_plot=2, epoch='end', batch_id='end', save_dir=save_dir)
    break

FileNotFoundError: ignored